In [2]:
%matplotlib inline

from scipy import interpolate
import numpy as np
import cgs as cgs
import dwarf_model as dw_model
from initial_conditions import ic_list as icl ;
from matplotlib import rc
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
import sys
from __future__ import print_function

using cgs py
Loading IC for  Leo_T_burkert
Loading IC for  Leo_T
MUST SET A 'potential_type'
Loading IC for  Sextans_test
MUST SET A 'potential_type'
Loading IC for  Leo_T_obs
Assuming primordial neutral for dwarf mu = 1.31
Assuming primordial ionized for halo mu = 0.6
If M200 or n_o are not set, they will be solved for
using the profile choice
Loading IC for  CarinaMidMed
If M200 or n_o are not set, they will be solved for
using the profile choice
Loading IC for  Leo_T_solve_burkert
Assuming primordial neutral for dwarf mu = 1.31
Assuming primordial ionized for halo mu = 0.6
If M200 or n_o are not set, they will be solved for
using the profile choice
Loading IC for  Leo_test
MUST SET A 'potential_type'
Loaded IC's for  7  dwarf galaxies


In [3]:
carina = icl.ic_object_dict['CarinaMidMed']

carina_orbit = np.genfromtxt("./../orbits/carina_orbit_tab.dat")
t_orbit = carina_orbit[:,0] * cgs.Myr
v_orbit = carina_orbit[:,2] * 1.0E5
rho_halo = carina.ic['n_halo'] * cgs.mp * carina.ic['mu_halo']

anal_carina = dw_model.analytical_dwarf('CarinaMidMed',carina.ic)
anal_carina.setup_orbit(t_orbit,rho_halo,v_orbit)

In [4]:
anal_carina.ic

{'M200': 6.74000417704841e+41,
 'M_DM': 7.3596700000000005e+40,
 'RM': 1.3056121310209298e+21,
 'T_dwarf': 10000.0,
 'T_halo': 1800000.0,
 'b': 2.4531136761000005e+21,
 'c': 17.753195173656437,
 'mu_dwarf': 1.297,
 'mu_halo': 0.62,
 'n_halo': 0.00017,
 'n_o': 0.544,
 'potential_type': 'NFW',
 'r_DM': 2.6845394946e+21,
 'r_s': 2.4531136761000005e+21,
 'rho_crit': 9.74e-30}

In [5]:
fpath="/home/emerick/Research/dwarfs/flash_runs/"
adiabatic = np.genfromtxt(fpath + "carina_adiabatic/carina_adiabatic_mass_dt5Myr.dat",names=True)
adiabatic['t'] = adiabatic['t'] * cgs.Myr
adiabatic['m'] = adiabatic['m'] * cgs.Msun

In [6]:

def function_to_fit(x, p0, p1, p2, dwarf, method):

    def _mass_function(a,rpsb,khb,t,d,m):
        M_fit, R_fit = d.evolve(t, ['RPS','KH'],
                                   physics_kwargs={'RPS':{'alpha':a,'beta':rpsb, 'method':m,
                                                                          'T_galaxy':d.ic['T_dwarf'],
                                                                          'mu_galaxy':d.ic['mu_dwarf']},
                                                   'KH':{'beta':khb}})
        return M_fit
    # parameters for fit
    alpha     = p0
    rps_beta  = p1
    kh_beta   = p2
    
    t = np.linspace(0.0,np.max(t_orbit)/cgs.Myr,2000.0)*cgs.Myr
    # evolve to get m as function of time
    m = _mass_function(alpha,rps_beta,kh_beta,t,dwarf,method)
    
    # recast as callable function
    function = interpolate.UnivariateSpline(t, m,k=3)
    
    return function(x)

In [7]:
#func = lambda x,alpha,beta: function_to_fit(x,alpha,beta,anal_carina,method='shock')

#popt, pconv = curve_fit(func, adiabatic['m'], adiabatic['t'], p0=[0.75,25.0])

In [8]:
# time to do some manual fitting
alphas     = np.linspace(0.5, 2.5,12)
rps_betas  = np.linspace(0.1, 1.5,12)
kh_betas   = np.linspace(0.1, 1.5,12)

In [ ]:
%%capture --no-stderr

nparams = np.size(alphas)*np.size(rps_betas)*np.size(kh_betas)
error  = np.zeros(nparams)
params = np.array([[0.,0.,0.]]*nparams)*1.0
i=0
f = open('shock_parameter_study.dat','w')
f.write("#alpha rps_beta kh_beta error\n")
out_format = "%8.8E %8.8E %8.8E %8.8E\n"
for a in alphas:
    for rps_b in rps_betas:
        for kh_b in kh_betas:
            m = function_to_fit(adiabatic['t'],a,rps_b,kh_b,anal_carina,method='shock');
        
            error[i] = np.sum((m - adiabatic['m'])**2)
        
            params[i][0] = a * 1.0 
            params[i][1] = rps_b * 1.0
            params[i][2] = kh_b*1.0
            print('%i of %i - alpha = %.3f , rps_beta = %.3f, kh_beta %.3f, error = %.5e'%(i+1,nparams,a,rps_b,kh_b,error[i]),file=sys.stderr)
            f.write(out_format%(a,rps_b,kh_b,error[i]))
            i = i + 1
        
f.close()        
params = np.array(params)


/home/emerick/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack2.py:188: UserWarning: 
A theoretically impossible result was found during the iteration
proces for finding a smoothing spline with fp = s: s too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
1 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta 0.100, error = 7.41184e+78
2 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta 0.227, error = 5.61466e+78
3 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta 0.355, error = 4.16687e+78
4 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta 0.482, error = 3.02538e+78
5 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta 0.609, error = 2.15401e+78
6 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta 0.736, error = 1.52254e+78

In [40]:
print(np.argmin(error), np.min(error) , np.median(error))
print(params[np.argmin(error)])

min_err_index = np.argmin(error)
best_alpha = params[min_err_index][0]
best_beta  = params[min_err_index][1]
string = "best alpha  = %.4f - best beta = %.4f\n"%(best_alpha,best_beta)
print(string)
outfile = open('outfile.dat','w')
outfile.write('shock\n')
outfile.write(string)

12 6.05923450504e+77 8.33774207818e+78
[ 0.5         0.18181818  0.1       ]
best alpha  = 0.5000 - best beta = 0.1818



In [ ]:
%%capture --no-stderr

nparams = np.size(alphas)*np.size(rps_betas)*np.size(kh_betas)
sound_error  = np.zeros(nparams)
sound_params = np.array([[0.,0.,0.]]*nparams)*1.0
i=0
f = open('sound_parameter_study.dat','w')
f.write("#alpha rps_beta kh_beta error\n")
out_format = "%8.8E %8.8E %8.8E %8.8E\n"
for a in alphas:
    for rps_b in rps_betas:
        for kh_b in kh_betas:
            m = function_to_fit(adiabatic['t'],a,rps_b,kh_b,anal_carina,method='sound');
        
            sound_error[i] = np.sum((m - adiabatic['m'])**2)
        
            sound_params[i][0] = a * 1.0 
            sound_params[i][1] = rps_b * 1.0
            sound_params[i][2] = kh_b*1.0
            print('%i of %i - alpha = %.3f , rps_beta = %.3f, kh_beta = %.3f, error = %.5e'%(i+1,nparams,a,rps_b,kh_b,sound_error[i]),file=sys.stderr)
            f.write(out_format%(a,rps_b,kh_b,error[i]))
            i = i + 1
f.close()        
sound_params = np.array(sound_params)


1 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 0.100, error = 4.80482e+152
2 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 0.545, error = 2.53393e+155
3 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 0.991, error = 2.40015e+151
4 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 1.436, error = 1.20197e+155
5 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 1.882, error = 4.98475e+154
6 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 2.327, error = 3.50021e+155
7 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 2.773, error = 4.20273e+155
8 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 3.218, error = 2.16344e+169
9 of 1728 - alpha = 0.500 , rps_beta = 0.100, kh_beta = 3.664, error = 1.70150e+170

In [ ]:
print(np.argmin(sound_error), np.min(sound_error) , np.median(sound_error))
print(sound_params[np.argmin(sound_error)])

min_err_index = np.argmin(sound_error)
best_sound_alpha = sound_params[min_err_index][0]
best_sound_beta  = sound_params[min_err_index][1]

string = "best alpha  = %.4f - best beta = %.4f\n"%(best_sound_alpha,best_sound_beta)
print(string)
outfile.write('sound\n')
outfile.write(string)

In [ ]:
alpha_shock = 0.5 #best_alpha
rps_beta_shock  = 0.18181818 #best_beta
kh_beta_shock = 0.1 #best_kh_beta
#alpha_sound = best_sound_alpha 0.5         0.18181818  0.1  
#beta_sound = best_sound_beta

m_shock = function_to_fit(adiabatic['t'],alpha_shock,rps_beta_shock,kh_beta_shock,anal_carina,method='shock')
#m_sound = function_to_fit(adiabatic['t'],alpha_sound,beta_sound,anal_carina,method='sound')

error_shock = np.sum(((m_shock-adiabatic['m'])/adiabatic['m'][0])**2)
#error_sound = np.sum(((m_sound-adiabatic['m'])/adiabatic['m'][0])**2)

plt.plot(adiabatic['t']/cgs.Myr, adiabatic['m']/adiabatic['m'][0],color='red',lw=2,label='sim')
plt.plot(adiabatic['t']/cgs.Myr, m_shock/m_shock[0] ,color='black',lw=2,label='model - shock')
#plt.plot(adiabatic['t']/cgs.Myr, m_sound/m_sound[0] ,color='blue',lw=2,label='model - sound')

plt.ylim(0,1)
plt.legend(loc='best')

In [ ]:
outfile.close()

In [ ]:
f = open('alpha_beta_rps.dat','w')
string = "%8.8e %8.8e %8.8e %8.8e %8.8e %8.8e\n"

f.write("alpha_shock beta_shock error_shock alpha_sound beta_sound error_sound\n")
for i in np.arange(np.size(alpha_shock)):
    f.write(string%(alpha_shock[i], beta_shock[i], error_shock[i], alpha_sound[i], beta_sound[i], error_sound[i]))
    
f.close()